### imports
***

In [ ]:
#!wget http://tianchi-public-us-east-download.oss-us-east-1.aliyuncs.com/231786/sample/multimodal_train_sampleset.zip
!wget http://tianchi-public-us-east-download.oss-us-east-1.aliyuncs.com/231786/multimodal_valid.zip
!wget http://tianchi-public-us-east-download.oss-us-east-1.aliyuncs.com/231786/multimodal_testA.zip
!wget http://tianchi-public-us-east-download.oss-us-east-1.aliyuncs.com/231786/multimodal_testB.zip

In [ ]:
#!unzip  multimodal_train_sampleset.zip
!unzip  multimodal_valid.zip
!unzip  multimodal_testA.zip
!unzip  multimodal_testB.zip


In [ ]:
import torch, math
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
from torch import optim
from torch.optim.lr_scheduler import LambdaLR

from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel

from gensim import corpora, similarities, models
from gensim.matutils import corpus2csc

import pandas as pd 
import numpy as np
from collections import Counter
import base64
from tqdm import tqdm
import pickle, random
import matplotlib.pyplot as plt
from multiprocessing import Pool
import sys, csv, json, os, gc, time

In [ ]:
import torch, math
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
from torch import optim
from torch.optim.lr_scheduler import LambdaLR

from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel

from gensim import corpora, similarities, models
from gensim.matutils import corpus2csc
from sklearn.model_selection import KFold
from lightgbm import LGBMClassifier

import pandas as pd 
import numpy as np
from collections import Counter
import base64
from tqdm.auto import tqdm
import pickle, random
import matplotlib.pyplot as plt
from multiprocessing import Pool
import sys, csv, json, os, gc, time

In [ ]:
testCode=False
if testCode:
    Data = pd.read_csv(r'./train.sample.tsv', sep='\t',error_bad_lines=False, quoting=csv.QUOTE_NONE, encoding='utf-8')
    Data.reset_index(inplace=True)
    for i in range(0, 20):
        Data[i*500 : (i+1)*500].to_pickle('./train_sample_{}.pkl'.format(i))
    del Data

In [ ]:
splitTrainData=False
if splitTrainData:
    Data = pd.read_csv(r'../data/train.tsv', sep='\t', quoting=csv.QUOTE_NONE, encoding='utf-8').sample(frac=0.1)
    Data.reset_index(inplace=True)
    for i in range(0, 20):
        Data[i*15000 : (i+1)*15000].to_pickle('../kaggleData/train_{}.pkl'.format(i))
    del Data

### parameters
***

In [ ]:
no = '0'
device = torch.device('cuda:'+no) if torch.cuda.is_available() else torch.device('cpu')
print(device)

k = 10
lr = 1e-5
# true batch size = batch_size * grad_step
batch_size = 64
margin = 8
grad_step = 1
max_lang_len = 15
max_img_len = 30
epochs = 1
MOD = 20000
shuffle_fold = True
seed = 9527
random.seed(seed)
torch.manual_seed(seed)

In [ ]:
class params:
    IMG_FEAT_SIZE = 2048+6
    WORD_EMBED_SIZE = 1024
    LAYER = 6
    HIDDEN_SIZE = 1024
    MULTI_HEAD = 8
    DROPOUT_R = 0.1
    FLAT_MLP_SIZE = 512
    FLAT_GLIMPSES = 1
    FLAT_OUT_SIZE = 2048
    FF_SIZE = HIDDEN_SIZE*4
    HIDDEN_SIZE_HEAD = HIDDEN_SIZE // MULTI_HEAD
    OPT_BETAS = (0.9, 0.98)
    OPT_EPS = 1e-9
    TRAIN_SIZE = 300000#三十万

__C = params()

### load data
***

In [ ]:
trash = {'!', '$', "'ll", "'s", ',', '&', ':', 'and', 'cut', 'is', 'are', 'was'}
trash_replace = ['"hey siri, play some', 'however, ', 'yin and yang, ',
                 'shopping mall/']

def process(x):
    tmp = x.split()
    if tmp[0] in trash: x = ' '.join(tmp[1:])
    if tmp[0][0] == '-': x = x[1:]
    for tr in trash_replace:
        x = x.replace(tr, '')
    return x

def normalize(x):
    ret = x['boxes'].copy()
    ret[:,0] /= x['image_h']
    ret[:,1] /= x['image_w']
    ret[:,2] /= x['image_h']
    ret[:,3] /= x['image_w']
    wh = (ret[:,2]-ret[:,0]) * (ret[:,3]-ret[:,1])
    wh2 = (ret[:,2]-ret[:,0]) / (ret[:,3]-ret[:,1]+1e-6)
    ret = np.hstack((ret, wh.reshape(-1,1), wh2.reshape(-1,1)))
    return ret

def sort_by_area(x):
    return np.array(sorted(x.tolist(), key=lambda x: x[-1], reverse=True))

def load_data(file_name, reset=False, decode=True,pickle=False):
    if pickle:
        ret=pd.read_pickle(file_name)
    else:
        ret = pd.read_csv(file_name, sep='\t')
    if decode:
        ret['boxes'] = ret['boxes'].apply(lambda x: np.frombuffer(base64.b64decode(x), dtype=np.float32).reshape(-1, 4))
        ret['features'] = ret['features'].apply(lambda x: np.frombuffer(base64.b64decode(x), dtype=np.float32).reshape(-1, 2048))
        ret['class_labels'] = ret['class_labels'].apply(lambda x: np.frombuffer(base64.b64decode(x), dtype=np.int64).reshape(-1, 1))
        ret['boxes'] = ret.apply(lambda x: normalize(x), axis=1)
        ret['features'] = ret.apply(lambda x: np.concatenate((x['features'], x['boxes']), axis=1)[:max_img_len], axis=1)
    ret['query'] = ret['query'].apply(lambda x: process(x))
    # reset query_id
    if reset:
        query2qid = {query: qid for qid, (query, _) in enumerate(tqdm(ret.groupby('query')))}
        ret['query_id'] = ret.apply(lambda x: query2qid[x['query']], axis=1)
    return ret

In [ ]:
path = '../input/graduationkaggle/'
validDf = load_data('./valid.tsv')
testADf = load_data('./testA.tsv')
testBDf = load_data('./testB.tsv')
validAnswer = json.loads(open('./valid_answer.json', 'r').read())
testAAnswer = json.loads(open('../input/graduationtesta-banswer/testA_answer.json', 'r').read())
testBAnswer = json.loads(open('../input/graduationtesta-banswer/testB_answer.json', 'r').read())

### preprocess
***

In [ ]:
# load pre-trained model
take = 'bert-large-uncased'
emb_size = __C.WORD_EMBED_SIZE
tokenizer = AutoTokenizer.from_pretrained(take)
pretrained_emb = AutoModel.from_pretrained(take)
pad_id = tokenizer.pad_token_id
sep_id = tokenizer.sep_token_id

qid2token = {qid: tokenizer.encode(group['query'].values[0]) for qid, group in tqdm(testADf.groupby('query_id'))}
testADf['token'] = testADf['query_id'].apply(lambda x: qid2token[x])
testADf['token'] = testADf['token'].apply(lambda x: x[:max_lang_len-1]+[sep_id] if len(x) > max_lang_len else x)

qid2token = {qid: tokenizer.encode(group['query'].values[0]) for qid, group in tqdm(testBDf.groupby('query_id'))}
testBDf['token'] = testBDf['query_id'].apply(lambda x: qid2token[x])
testBDf['token'] = testBDf['token'].apply(lambda x: x[:max_lang_len-1]+[sep_id] if len(x) > max_lang_len else x)

qid2token = {qid: tokenizer.encode(group['query'].values[0]) for qid, group in tqdm(validDf.groupby('query_id'))}
validDf['token'] = validDf['query_id'].apply(lambda x: qid2token[x])
validDf['token'] = validDf['token'].apply(lambda x: x[:max_lang_len-1]+[sep_id] if len(x) > max_lang_len else x)

In [ ]:
testBDf['target'] = testBDf.apply(lambda x: 1 if x['product_id'] in testBAnswer[str(x['query_id'])] else 0, axis=1)
testADf['target'] = testADf.apply(lambda x: 1 if x['product_id'] in testAAnswer[str(x['query_id'])] else 0, axis=1)
validDf['target'] = validDf.apply(lambda x: 1 if x['product_id'] in validAnswer[str(x['query_id'])] else 0, axis=1)

### training data
***

In [ ]:
def one(x):
    return (x+1e-10)/(x+1e-10)

def get_negs(train):
    qid2idxs = {}
    corpus = []
    idx2qid = {}

    for idx, (qid, group) in enumerate(train.groupby('query_id')):
        qid2idxs[qid] = group.index
        corpus.append(group['query'].values[0])
        idx2qid[idx] = qid

    topk = len(max(qid2idxs.values(), key=lambda x: len(x)))*3
    corpus = [sent.split() for sent in corpus]
    dictionary = corpora.Dictionary(corpus)
    corpus = [dictionary.doc2bow(text) for text in corpus]
    tfidf_model = models.TfidfModel(corpus, wlocal=one, dictionary=dictionary)
    corpus_tfidf = corpus2csc(tfidf_model[corpus])
    sm = corpus_tfidf.T.dot(corpus_tfidf)
    qid2negs = {}
    
    for idx, (le, ri) in enumerate(tqdm(zip(sm.indptr[:-1], sm.indptr[1:]), total=sm.shape[0])):
        n_row_pick = min(topk, ri-le)
        top_n_idx = sm.indices[le+np.argpartition(sm.data[le:ri], -n_row_pick)[-n_row_pick:]].tolist()
        if n_row_pick < topk: top_n_idx += random.sample(range(sm.shape[0]), topk-n_row_pick)
        qid2negs[idx2qid[idx]] = [idx2qid[neg] for neg in top_n_idx if neg != idx]
        
    return qid2negs, qid2idxs

In [ ]:
def get_data(train, qid2negs, qid2idxs):
    train_x = [] # [tokens, feature1, feature2]
    
    for qid, group in tqdm(train.groupby('query_id')):
        # positive
        pos = group.index
        token = group['token'].iloc[0]
        # negative
        for i in range(k):
            neg = random.sample(qid2negs[qid], len(pos))
            neg = [random.choice(qid2idxs[n]) for n in neg]
            train_x += [[token, pos[i], neg[i]] for i in range(len(pos))]
    
    print('number of training data:', len(train_x))
    return train_x

### model
***

In [ ]:
class FC(nn.Module):
    def __init__(self, in_size, out_size, dropout_r=0., use_relu=True):
        super(FC, self).__init__()
        self.dropout_r = dropout_r
        self.use_relu = use_relu

        self.linear = nn.Linear(in_size, out_size)
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(dropout_r)

    def forward(self, x):
        x = self.linear(x)
        x = self.relu(x)
        x = self.dropout(x)
        return x


class MLP(nn.Module):
    def __init__(self, in_size, mid_size, out_size, dropout_r=0., use_relu=True):
        super(MLP, self).__init__()

        self.fc = FC(in_size, mid_size, dropout_r=dropout_r, use_relu=use_relu)
        self.linear = nn.Linear(mid_size, out_size)

    def forward(self, x):
        return self.linear(self.fc(x))


class VisualBERT(nn.Module):
    def __init__(self, __C, bert):
        super(VisualBERT, self).__init__()
        
        self.bert = bert
        self.linear_img = nn.Linear(__C.IMG_FEAT_SIZE, __C.WORD_EMBED_SIZE)
        self.out = MLP(__C.WORD_EMBED_SIZE, __C.WORD_EMBED_SIZE//2, 1)
        
    def forward(self, ques_ix, img_feats):
        proj_feats = []
        for img_feat in img_feats:
            # Make mask & token type ids
            mask = self.make_mask(ques_ix.unsqueeze(2), img_feat, pad_id)
            token = self.get_token_type(ques_ix, img_feat)
            # Preprocess features
            lang_feat = self.bert.embeddings.word_embeddings(ques_ix)
            img_feat = self.linear_img(img_feat)
            combine_feat = torch.cat((lang_feat, img_feat), dim=1)
            # Token embeddings & position embeddings
            position_ids = torch.arange(token.size(1), dtype=torch.long, device=device)
            position_ids = position_ids.unsqueeze(0).expand(token.size())
            position_embeddings = self.bert.embeddings.position_embeddings(position_ids)
            token_type_embeddings = self.bert.embeddings.token_type_embeddings(token)
            # Add all
            embeddings = combine_feat+position_embeddings+token_type_embeddings
            embeddings = self.bert.embeddings.dropout(self.bert.embeddings.LayerNorm(embeddings))
            # Go through the rest of BERT
            head_mask = self.bert.get_head_mask(None, self.bert.config.num_hidden_layers)
            extended_attention_mask = self.bert.get_extended_attention_mask(mask, mask.size(), device)
            encoder_outputs = self.bert.encoder(embeddings,
                                                attention_mask=extended_attention_mask,
                                                head_mask=head_mask,
                                                encoder_hidden_states=None,
                                                encoder_attention_mask=None)
            # CLS embedding & output value
            outputs = encoder_outputs[0][:,0,:]
            proj_feats.append(self.out(outputs))
        return proj_feats
            
    # Masking
    def make_mask(self, lang_feat, img_feat, target):
        # 1 for NOT masked; 0 for masked
        # [batch, len]
        lang_mask = (torch.sum(torch.abs(lang_feat), dim=-1) != target).long()
        img_mask = (torch.sum(torch.abs(img_feat), dim=-1) != 0).long()
        return torch.cat((lang_mask, img_mask), dim=1)
    
    # Token type ids
    def get_token_type(self, lang_feat, img_feat):
        #    lang      img
        # 0 0 0 0 0 0 1 1 1 1 1
        lang_token = torch.zeros(lang_feat.size(0), lang_feat.size(1)).to(device)
        img_token = torch.ones(img_feat.size(0), img_feat.size(1)).to(device)
        return torch.cat((lang_token, img_token), dim=1).long()

### train
***

In [ ]:
def predict(model,test):
    model = model.eval()
    preds = {}
    
    with torch.no_grad():
        for qid, group in tqdm(test.groupby('query_id')):
            # prepare batch
            tokens, features = group['token'].values.tolist(), group['features'].values.tolist()
            max_len_f = len(max(features, key=lambda x: len(x)))
            features = [np.concatenate((feature, np.zeros((max_len_f-feature.shape[0], feature.shape[1]))), axis=0) for feature in features]
            # # to tensor
            tokens = torch.LongTensor(tokens).to(device)
            features = torch.FloatTensor(features).to(device)
            # predict
            out = model(tokens, (features,))[0].view(-1)
            pred = [(pid, val) for pid, val in zip(group['product_id'].values.tolist(), out.tolist())]
            pred.sort(key=lambda x: x[1], reverse=True)
            preds[qid] = [pid for pid, _ in pred]
            
    model = model.train()
    return preds

In [ ]:
class CustomDataset(data.Dataset):
    def __init__(self, train_x):
        self.train_x = train_x
        
    def __getitem__(self, index):
        tokens, pos_features, neg_features = self.train_x[index][0], self.train_x[index][1], self.train_x[index][2]
        return [tokens, pos_features, neg_features]
    
    def __len__(self):
        return len(self.train_x)
    
def collate_fn(batch):
    tokens, pos_features, neg_features = zip(*batch)
    max_len_t, max_len_pf, max_len_nf = len(max(tokens, key=lambda x: len(x))), len(max(pos_features, key=lambda x: len(x))), len(max(neg_features, key=lambda x: len(x)))
    tokens, pos_features, neg_features = [token+[pad_id]*(max_len_t-len(token)) for token in tokens], [np.concatenate((feature, np.zeros((max_len_pf-feature.shape[0], feature.shape[1]))), axis=0) for feature in pos_features], [np.concatenate((feature, np.zeros((max_len_nf-feature.shape[0], feature.shape[1]))), axis=0) for feature in neg_features]
    return torch.LongTensor(tokens), torch.FloatTensor(pos_features), torch.FloatTensor(neg_features)

def custom_schedule(optimizer, num_warmup_steps, num_training_steps, num_cycles=0.5, amplitude=0.1, last_epoch=-1):
    
    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        progress = 2.0 * math.pi * float(num_cycles) * float(current_step-num_warmup_steps) / float(max(1, num_training_steps-num_warmup_steps))
        linear = float(num_training_steps-current_step) / float(max(1, num_training_steps-num_warmup_steps))
        return abs(linear + math.sin(progress)*linear*amplitude)

    return LambdaLR(optimizer, lr_lambda, last_epoch)

def shuffle(x):
    idxs = [i for i in range(x.shape[0])]
    random.shuffle(idxs)
    return x[idxs]

def nDCG_score(preds, answers):
    iDCG = sum([sum([np.log(2)/np.log(i+2) for i in range(min(len(answer), 5))]) \
                for answer in list(answers.values())])
    DCG = sum([sum([np.log(2)/np.log(i+2) if preds[qid][i] in answers[str(qid)] else 0 \
                    for i in range(len(preds[qid]))]) for qid in list(preds.keys())])
    return DCG/iDCG

class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, logits=False, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduce = reduce

    def forward(self, inputs, targets):
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduce=False)
        else:
            BCE_loss = F.binary_cross_entropy(inputs, targets, reduce=False)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss

In [ ]:
def get_data_merge(idx):
#     train = load_data(path+'train.sample.tsv', reset=True)
    if testCode:
        train = load_data(path+'train_sample_{}.pkl'.format(idx), reset=True, decode=False,pickle=True)
    else:
        train = load_data(path+'train_{}.pkl'.format(idx), reset=True, decode=False,pickle=True)
    train.reset_index(inplace=True)
    qid2token = {qid: tokenizer.encode(group['query'].values[0]) for qid, group in train.groupby('query_id')}
    train['token'] = train['query_id'].apply(lambda x: qid2token[x])
    qid2negs, qid2idxs = get_negs(train)
    if testCode:
        pickle.dump(get_data(train, qid2negs, qid2idxs),
                    open(path+'data/train_sample_x_{}_{}.pkl'.format(idx, seed), 'wb'))
    else:
        pickle.dump(get_data(train, qid2negs, qid2idxs),
                    open(path+'data/train_x_{}_{}.pkl'.format(idx, seed), 'wb'))
    return None

def load_features(idx):
    print('load  data idx ---->:',idx)
#     train = load_data(path+'train.sample.tsv', reset=True)
    if testCode:
        train = load_data(path+'train_sample_{}.pkl'.format(idx), reset=True,pickle=True)
        train_x = pickle.load(open(path+'train_sample_x_{}_{}.pkl'.format(idx, seed), 'rb'))
    else:
        train = load_data(path+'train_{}.pkl'.format(idx), reset=True,pickle=True)
        train_x = pickle.load(open(path+'train_x_{}_{}.pkl'.format(idx, seed), 'rb'))
    train.reset_index(inplace=True)
    train_x = [[t, train['features'].iloc[p], train['features'].iloc[n]] \
               for t, p, n in train_x]
    return train_x

def get_data_chunk(start, end):
    ret = []
    for fold in range(start, end):
        t0 = time.time()
        print('reading fold:', fold)

        try:
            temp=load_features(fold_idxs[fold])
        except:
            print(fold_idxs[fold],' is wrong !!!!!!!!!!!!!!!!!!!')
        else:
            ret+=temp
        t = round((time.time()-t0))
        print('time consumed: {} min {} sec'.format(t//60, t%60))
    return ret

In [ ]:
reduceDataset=False
if reduceDataset:
    t0 = time.time()
    with Pool(5) as pool:
        pool.map(get_data_merge, [i for i in range(20)])
    t = round((time.time()-t0)/60)
    print('time consumed: {} hr {} min'.format(t//60, t%60))

In [ ]:
print('initializing model...')
nDCGs=[]
best_nDCG = 0.0
model = VisualBERT(__C, pretrained_emb).to(device)
num_training_steps = np.ceil(__C.TRAIN_SIZE*k / (batch_size*grad_step)) * epochs
num_warmup_steps = int(num_training_steps*0.1)
eval_steps = num_training_steps//40*grad_step + 100
optimizer = AdamW(model.parameters(), lr=lr, correct_bias=False)
scheduler = custom_schedule(optimizer,
                            num_warmup_steps=num_warmup_steps,
                            num_training_steps=num_training_steps,
                            num_cycles=6,
                            amplitude=0.3)

In [ ]:
model.load_state_dict(torch.load('../input/graduationkagglemedmodal/model_Visual-BERT_pair_box_tfidf-neg_focal_all_7328_39',map_location=device))
model=model.to(device)

In [ ]:
import gc
def train_empohs():
    global model
    for chunk in range(5):
        train_x_all = get_data_chunk(chunk*3, min((chunk+1)*3,20))
        gc.collect()
        random.shuffle(train_x_all)
        steps = (min((chunk+1)*3, 20) - chunk*3) * 2
        step_size = len(train_x_all)//steps

        for step in range(steps):
            train_loader = data.DataLoader(CustomDataset(train_x_all[step_size*step:step_size*(step+1)]),
                                           batch_size=batch_size,
                                           shuffle=True,
                                           collate_fn=collate_fn,
                                           num_workers=4)
            model = model.train()
            criterion = FocalLoss()
            total_loss = 0.0
            step = len(train_loader)
            optimizer.zero_grad()
            pbar = tqdm(enumerate(train_loader), total=step)

            for i, batch in pbar:
                # prepare batch
                tokens, pos_features, neg_features = batch
                # # to device
                tokens = tokens.to(device)
                pos_features = pos_features.to(device)
                neg_features = neg_features.to(device)
                # predict
                pos, neg = model(tokens, (pos_features, neg_features))
                pos = torch.sigmoid(pos).view(-1)
                neg = torch.sigmoid(neg).view(-1)
                l = criterion(pos, torch.ones(pos.size()).to(device))
                l.backward()
                total_loss += l.item()
                l = criterion(neg, torch.zeros(neg.size()).to(device))
                l.backward()
                total_loss += l.item()
                pbar.set_postfix({'loss': total_loss/(i+1)})
                # optim step
                if (i+1)%grad_step == 0:
                    optimizer.step()
                    scheduler.step()
                    optimizer.zero_grad()

            if step%grad_step:
                optimizer.step()
                scheduler.step()
            # delete garbage
            del train_loader
        del train_x_all
        gc.collect()

In [ ]:
def getNDCG(preds, answers,k):
    iDCG = sum([sum([np.log(2)/np.log(i+2) for i in range(min(len(answer), k))]) \
                for answer in list(answers.values())])
    DCG = sum([sum([np.log(2)/np.log(i+2) if preds[qid][i] in answers[str(qid)] else 0 \
                    for i in range(min(len(preds[qid]), k))]) for qid in list(preds.keys())])
    return DCG/iDCG

In [ ]:
def Recall(pred,target,k):
    k=min(len(pred), k)
    n = len( np.intersect1d(pred[:k],target))
    if len(target)>0:
        return n / len(target)
    else:
        return 0
def getRecall(preds,targets,k):
    scores=[]
    for query in list(preds.keys()):
        scores.append(Recall(preds[query],targets[str(query)],k))
    return np.mean(scores)

In [ ]:
def Precision(pred,target,k):
    k=min(len(pred), k)
    n = len( np.intersect1d(pred[:k],target))
    if k>0:
        return n / k
    else:
        return 0
def getPrecision(preds,targets,k):
    scores=[]
    for query in list(preds.keys()):
        scores.append(Precision(preds[query],targets[str(query)],k))
    return np.mean(scores)

In [ ]:
def getF1(preds,targets,k):
    scores=[]
    for query in list(preds.keys()):
        p=Precision(preds[query],targets[str(query)],k)
        r=Recall(preds[query],targets[str(query)],k)
        if p+r>0:
            f1=(2*p*r)/(p+r)
        else:
            f1=0
        scores.append(f1)
    return np.mean(scores)

In [ ]:
# coding=utf-8

import numpy as np
import matplotlib.pyplot as plt
def plotCurve(x,y):
    plt.rcParams['font.sans-serif'] = ['Arial']  # 如果要显示中文字体,则在此处设为：SimHei
    plt.rcParams['axes.unicode_minus'] = False  # 显示负号

    x = np.array(x)
    y = np.array(y)
    plt.figure(figsize=(10, 5))
    plt.grid(linestyle="--")  # 设置背景网格线为虚线
    ax = plt.gca()
    ax.spines['top'].set_visible(False)  # 去掉上边框
    ax.spines['right'].set_visible(False)  # 去掉右边框


    plt.plot(x, y, marker='o', color="blue", label="unpretrain", linewidth=1.5)
    plt.xlabel("epochs", fontsize=13, fontweight='bold')
    plt.ylabel("nDCG@5 on testB", fontsize=13, fontweight='bold')
    plt.xlim(0, 10)  # 设置x轴的范围
    plt.ylim(0, 0.6)

  # plt.legend()          #显示各曲线的图例
    plt.legend(loc=0, numpoints=1)
    leg = plt.gca().get_legend()
    ltext = leg.get_texts()
    plt.setp(ltext, fontsize=12, fontweight='bold')  # 设置图例字体的大小和粗细
    plt.show()


In [ ]:
def extract_embedding(model, ques_ix, img_feat):
    # Make mask & token type ids
    mask = model.make_mask(ques_ix.unsqueeze(2), img_feat, pad_id)
    token = model.get_token_type(ques_ix, img_feat)
    # Preprocess features
    lang_feat = model.bert.embeddings.word_embeddings(ques_ix)
    img_feat = model.linear_img(img_feat)
    combine_feat = torch.cat((lang_feat, img_feat), dim=1)
    # Token embeddings & position embeddings
    position_ids = torch.arange(token.size(1), dtype=torch.long, device=device)
    position_ids = position_ids.unsqueeze(0).expand(token.size())
    position_embeddings = model.bert.embeddings.position_embeddings(position_ids)
    token_type_embeddings = model.bert.embeddings.token_type_embeddings(token)
    # Add all
    embeddings = combine_feat+position_embeddings+token_type_embeddings
    embeddings = model.bert.embeddings.dropout(model.bert.embeddings.LayerNorm(embeddings))
    # Go through the rest of BERT
    head_mask = model.bert.get_head_mask(None, model.bert.config.num_hidden_layers)
    extended_attention_mask = model.bert.get_extended_attention_mask(mask, mask.size(), device)
    encoder_outputs = model.bert.encoder(embeddings,
                                         attention_mask=extended_attention_mask,
                                         head_mask=head_mask,
                                         encoder_hidden_states=None,
                                         encoder_attention_mask=None)
    # CLS embedding & output value
    outputs = encoder_outputs[0][:,0,:]
    return outputs

In [ ]:
def get_cls(model, n_splits,test):
    model.eval()
    qids = [[qid] for qid, _ in test.groupby('query_id')]
    kf = KFold(n_splits=n_splits, shuffle=True)
    train_x, train_y = [], []
    test_x, test_y = [], []
    qid2fold = {qids[idx][0]: i \
                for i, (train_index, test_index) in enumerate(kf.split(qids)) \
                for idx in test_index}
    
    with torch.no_grad():
        for qid, group in tqdm(test.groupby('query_id')):
            # prepare batch
            tokens, features = group['token'].values.tolist(), group['features'].values.tolist()
            max_len_f = len(max(features, key=lambda x: len(x)))
            features = [np.concatenate((feature, np.zeros((max_len_f-feature.shape[0], feature.shape[1]))), axis=0) for feature in features]
            # # to tensor
            tokens = torch.LongTensor(tokens).to(device)
            features = torch.FloatTensor(features).to(device)
            # predict
            tmp_x = extract_embedding(model, tokens, features).tolist()
            tmp_y = group['target'].values.tolist()
            # only use first fold
            if qid2fold[qid]:
                train_x += tmp_x
                train_y += tmp_y
            else:
                test_x += tmp_x
                test_y += tmp_y
    
    train_x, train_y = np.array(train_x), np.array(train_y)
    test_x, test_y = np.array(test_x), np.array(test_y)
    print('train:test = {}:{}'.format(train_x.shape[0], test_x.shape[0]))
    cls = LGBMClassifier(random_state=0, n_jobs=24)
    cls.fit(train_x, train_y,
            eval_set=[(test_x, test_y)],
            early_stopping_rounds=100,
            verbose=100)
        
    return cls

In [ ]:
def predictCLS(model, test, thd, cls):
    model.eval()
    counts = Counter(test['product_id'].values.tolist())
    preds = {}
    
    with torch.no_grad():
        for qid, group in tqdm(test.groupby('query_id')):
            # prepare batch
            tokens, features = group['token'].values.tolist(), group['features'].values.tolist()
            max_len_f = len(max(features, key=lambda x: len(x)))
            features = [np.concatenate((feature, np.zeros((max_len_f-feature.shape[0], feature.shape[1]))), axis=0) for feature in features]
            # # to tensor
            tokens = torch.LongTensor(tokens).to(device)
            features = torch.FloatTensor(features).to(device)
            # predict
            embeddings = np.array(extract_embedding(model, tokens, features).tolist())
            out = cls.predict_proba(embeddings)[:,1]
            pred = [(pid, val) for pid, val in zip(group['product_id'].values.tolist(), out.tolist())]
            thd_tmp = thd
            pred2 = [x for x in pred]
            pred2.sort(key=lambda x: x[1], reverse=True)
            preds[qid] = [pid for pid, _ in pred2]
            
    return preds

In [ ]:
def predict_all(model, test, pad_len, cls):
    model.eval()
    preds = {}
    
    with torch.no_grad():
        for qid, group in tqdm(test.groupby('query_id')):
            # prepare batch
            tokens, features = group['token'].values.tolist(), group['features'].values.tolist()
            max_len_f = len(max(features, key=lambda x: len(x)))
            features = [np.concatenate((feature, np.zeros((max_len_f-feature.shape[0], feature.shape[1]))), axis=0) for feature in features]
            # # to tensor
            tokens = torch.LongTensor(tokens).to(device)
            features = torch.FloatTensor(features).to(device)
            # predict
            embeddings = np.array(extract_embedding(model, tokens, features).tolist())
            out = cls.predict_proba(embeddings)[:,1]
            pred = [(pid, val) for pid, val in zip(group['product_id'].values.tolist(), out.tolist())]
            pred.sort(key=lambda x: x[1], reverse=True)
            assert len(pred) <= pad_len
            pid, score = [p for p, s in pred], [s for p, s in pred]
            pid, score = pid+[np.nan]*(pad_len-len(pred)), score+[np.nan]*(pad_len-len(pred))
            preds[qid] = pid+score
            
    return preds

In [ ]:
#-------------------------------------------

In [ ]:
!mkdir ./content/
!mkdir ./content/drive/
!mkdir ./content/drive/MyDrive/
!mkdir ./content/drive/MyDrive/baseline_bertBert/
!mkdir ./content/drive/MyDrive/baseline_bertBert_lightGBM/

In [ ]:
import pickle
recallA=[]
precisionA=[]
f1A=[]
ndcgA=[]

recallB=[]
precisionB=[]
f1B=[]
ndcgB=[]

recallValid=[]
precisionValid=[]
f1Valid=[]
ndcgValid=[]

#------------------------
LrecallA=[]
LprecisionA=[]
Lf1A=[]
LndcgA=[]

LrecallB=[]
LprecisionB=[]
Lf1B=[]
LndcgB=[]

LrecallValid=[]
LprecisionValid=[]
Lf1Valid=[]
LndcgValid=[]
pad_len = 30
thd = 9999 
n_splits = 10

def saveFile():
    # 存储变量的文件的名字
    filename = './content/drive/MyDrive/baseline_bertBert/recallA.pkl'
    # 以二进制写模式打开目标文件
    f = open(filename, 'wb')
    # 将变量存储到目标文件中区
    pickle.dump(recallA, f)
    # 关闭文件
    f.close()

    # 存储变量的文件的名字
    filename = './content/drive/MyDrive/baseline_bertBert/precisionA.pkl'
    # 以二进制写模式打开目标文件
    f = open(filename, 'wb')
    # 将变量存储到目标文件中区
    pickle.dump(precisionA, f)
    # 关闭文件
    f.close()

    # 存储变量的文件的名字
    filename = './content/drive/MyDrive/baseline_bertBert/f1A.pkl'
    # 以二进制写模式打开目标文件
    f = open(filename, 'wb')
    # 将变量存储到目标文件中区
    pickle.dump(f1A, f)
    # 关闭文件
    f.close()


    # 存储变量的文件的名字
    filename = './content/drive/MyDrive/baseline_bertBert/ndcgA.pkl'
    # 以二进制写模式打开目标文件
    f = open(filename, 'wb')
    # 将变量存储到目标文件中区
    pickle.dump(ndcgA, f)
    # 关闭文件
    f.close()

    # 存储变量的文件的名字
    filename = './content/drive/MyDrive/baseline_bertBert/recallB.pkl'
    # 以二进制写模式打开目标文件
    f = open(filename, 'wb')
    # 将变量存储到目标文件中区
    pickle.dump(recallB, f)
    # 关闭文件
    f.close()

    # 存储变量的文件的名字
    filename = './content/drive/MyDrive/baseline_bertBert/precisionB.pkl'
    # 以二进制写模式打开目标文件
    f = open(filename, 'wb')
    # 将变量存储到目标文件中区
    pickle.dump(precisionB, f)
    # 关闭文件
    f.close()

    # 存储变量的文件的名字
    filename = './content/drive/MyDrive/baseline_bertBert/f1B.pkl'
    # 以二进制写模式打开目标文件
    f = open(filename, 'wb')
    # 将变量存储到目标文件中区
    pickle.dump(f1B, f)
    # 关闭文件
    f.close()

    # 存储变量的文件的名字
    filename = './content/drive/MyDrive/baseline_bertBert/ndcgB.pkl'
    # 以二进制写模式打开目标文件
    f = open(filename, 'wb')
    # 将变量存储到目标文件中区
    pickle.dump(ndcgB, f)
    # 关闭文件
    f.close()

    # 存储变量的文件的名字
    filename = './content/drive/MyDrive/baseline_bertBert/recallValid.pkl'
    # 以二进制写模式打开目标文件
    f = open(filename, 'wb')
    # 将变量存储到目标文件中区
    pickle.dump(recallValid, f)
    # 关闭文件
    f.close()

    # 存储变量的文件的名字
    filename = './content/drive/MyDrive/baseline_bertBert/precisionValid.pkl'
    # 以二进制写模式打开目标文件
    f = open(filename, 'wb')
    # 将变量存储到目标文件中区
    pickle.dump(precisionValid, f)
    # 关闭文件
    f.close()

    # 存储变量的文件的名字
    filename = './content/drive/MyDrive/baseline_bertBert/f1Valid.pkl'
    # 以二进制写模式打开目标文件
    f = open(filename, 'wb')
    # 将变量存储到目标文件中区
    pickle.dump(f1Valid, f)
    # 关闭文件
    f.close()

    # 存储变量的文件的名字
    filename = './content/drive/MyDrive/baseline_bertBert/ndcgValid.pkl'
    # 以二进制写模式打开目标文件
    f = open(filename, 'wb')
    # 将变量存储到目标文件中区
    pickle.dump(ndcgValid, f)
    # 关闭文件
    f.close()


    # 存储变量的文件的名字
    filename = './content/drive/MyDrive/baseline_bertBert_lightGBM/recallA.pkl'
    # 以二进制写模式打开目标文件
    f = open(filename, 'wb')
    # 将变量存储到目标文件中区
    pickle.dump(LrecallA, f)
    # 关闭文件
    f.close()

    # 存储变量的文件的名字
    filename = './content/drive/MyDrive/baseline_bertBert_lightGBM/precisionA.pkl'
    # 以二进制写模式打开目标文件
    f = open(filename, 'wb')
    # 将变量存储到目标文件中区
    pickle.dump(LprecisionA, f)
    # 关闭文件
    f.close()

    # 存储变量的文件的名字
    filename = './content/drive/MyDrive/baseline_bertBert_lightGBM/f1A.pkl'
    # 以二进制写模式打开目标文件
    f = open(filename, 'wb')
    # 将变量存储到目标文件中区
    pickle.dump(Lf1A, f)
    # 关闭文件
    f.close()


    # 存储变量的文件的名字
    filename = './content/drive/MyDrive/baseline_bertBert_lightGBM/ndcgA.pkl'
    # 以二进制写模式打开目标文件
    f = open(filename, 'wb')
    # 将变量存储到目标文件中区
    pickle.dump(LndcgA, f)
    # 关闭文件
    f.close()

    # 存储变量的文件的名字
    filename = './content/drive/MyDrive/baseline_bertBert_lightGBM/recallB.pkl'
    # 以二进制写模式打开目标文件
    f = open(filename, 'wb')
    # 将变量存储到目标文件中区
    pickle.dump(LrecallB, f)
    # 关闭文件
    f.close()

    # 存储变量的文件的名字
    filename = './content/drive/MyDrive/baseline_bertBert_lightGBM/precisionB.pkl'
    # 以二进制写模式打开目标文件
    f = open(filename, 'wb')
    # 将变量存储到目标文件中区
    pickle.dump(LprecisionB, f)
    # 关闭文件
    f.close()

    # 存储变量的文件的名字
    filename = './content/drive/MyDrive/baseline_bertBert_lightGBM/f1B.pkl'
    # 以二进制写模式打开目标文件
    f = open(filename, 'wb')
    # 将变量存储到目标文件中区
    pickle.dump(Lf1B, f)
    # 关闭文件
    f.close()

    # 存储变量的文件的名字
    filename = './content/drive/MyDrive/baseline_bertBert_lightGBM/ndcgB.pkl'
    # 以二进制写模式打开目标文件
    f = open(filename, 'wb')
    # 将变量存储到目标文件中区
    pickle.dump(LndcgB, f)
    # 关闭文件
    f.close()

    # 存储变量的文件的名字
    filename = './content/drive/MyDrive/baseline_bertBert_lightGBM/recallValid.pkl'
    # 以二进制写模式打开目标文件
    f = open(filename, 'wb')
    # 将变量存储到目标文件中区
    pickle.dump(LrecallValid, f)
    # 关闭文件
    f.close()

    # 存储变量的文件的名字
    filename = './content/drive/MyDrive/baseline_bertBert_lightGBM/precisionValid.pkl'
    # 以二进制写模式打开目标文件
    f = open(filename, 'wb')
    # 将变量存储到目标文件中区
    pickle.dump(LprecisionValid, f)
    # 关闭文件
    f.close()

    # 存储变量的文件的名字
    filename = './content/drive/MyDrive/baseline_bertBert_lightGBM/f1Valid.pkl'
    # 以二进制写模式打开目标文件
    f = open(filename, 'wb')
    # 将变量存储到目标文件中区
    pickle.dump(Lf1Valid, f)
    # 关闭文件
    f.close()

    # 存储变量的文件的名字
    filename = './content/drive/MyDrive/baseline_bertBert_lightGBM/ndcgValid.pkl'
    # 以二进制写模式打开目标文件
    f = open(filename, 'wb')
    # 将变量存储到目标文件中区
    pickle.dump(LndcgValid, f)
    # 关闭文件
    f.close()

In [ ]:
%%time
epochs = 1
for t in range(epochs):
    #train_empohs()
    global model
    testAPreds=predict(model,testADf)
    testBPreds=predict(model,testBDf)
    validPreds=predict(model,validDf)
    #recall
    rA=[]
    rB=[]
    rValid=[]
    for i in range(1,35,5):
        rA.append(getRecall(testAPreds,testAAnswer,i))
        rB.append(getRecall(testBPreds,testBAnswer,i))
        rValid.append(getRecall(validPreds,validAnswer,i))
    recallA.append(rA)
    recallB.append(rB)
    recallValid.append(rValid)

    #precision
    pA=[]
    pB=[]
    pValid=[]
    for i in range(1,35,5):
        pA.append(getPrecision(testAPreds,testAAnswer,i))
        pB.append(getPrecision(testBPreds,testBAnswer,i))
        pValid.append(getPrecision(validPreds,validAnswer,i))
    precisionA.append(pA)
    precisionB.append(pB)
    precisionValid.append(pValid)

    #f1
    f1a=[]
    f1b=[]
    f1valid=[]
    for i in range(1,35,5):
        f1a.append(getF1(testAPreds,testAAnswer,i))
        f1b.append(getF1(testBPreds,testBAnswer,i))
        f1valid.append(getF1(validPreds,validAnswer,i))
    f1A.append(f1a)
    f1B.append(f1b)
    f1Valid.append(f1valid)

    #ndcg
    ndcga=[]
    ndcgb=[]
    ndcgvalid=[]
    for i in range(1,8):
        ndcga.append(getNDCG(testAPreds,testAAnswer,i))
        ndcgb.append(getNDCG(testBPreds,testBAnswer,i))
        ndcgvalid.append(getNDCG(validPreds,validAnswer,i))
    ndcgA.append(ndcga)
    ndcgB.append(ndcgb)
    ndcgValid.append(ndcgvalid)
    y=[]
    for i in range(len(ndcgB)):
        y.append(ndcgB[i][-3])
    plotCurve(range(1,len(ndcgB)+1),y)
    #------------------------------------------------------
    cls = get_cls(model, n_splits,validDf)
        # train
    testAPreds=predictCLS(model, testADf, thd, cls)
    testBPreds=predictCLS(model, testBDf, thd, cls)
    validPreds=predictCLS(model, validDf, thd, cls)
    #recall
    rA=[]
    rB=[]
    rValid=[]
    for i in range(1,35,5):
        rA.append(getRecall(testAPreds,testAAnswer,i))
        rB.append(getRecall(testBPreds,testBAnswer,i))
        rValid.append(getRecall(validPreds,validAnswer,i))
    LrecallA.append(rA)
    LrecallB.append(rB)
    LrecallValid.append(rValid)

    #precision
    pA=[]
    pB=[]
    pValid=[]
    for i in range(1,35,5):
        pA.append(getPrecision(testAPreds,testAAnswer,i))
        pB.append(getPrecision(testBPreds,testBAnswer,i))
        pValid.append(getPrecision(validPreds,validAnswer,i))
    LprecisionA.append(pA)
    LprecisionB.append(pB)
    LprecisionValid.append(pValid)

    #f1
    f1a=[]
    f1b=[]
    f1valid=[]
    for i in range(1,35,5):
        f1a.append(getF1(testAPreds,testAAnswer,i))
        f1b.append(getF1(testBPreds,testBAnswer,i))
        f1valid.append(getF1(validPreds,validAnswer,i))
    Lf1A.append(f1a)
    Lf1B.append(f1b)
    Lf1Valid.append(f1valid)

    #ndcg
    ndcga=[]
    ndcgb=[]
    ndcgvalid=[]
    for i in range(1,8):
        ndcga.append(getNDCG(testAPreds,testAAnswer,i))
        ndcgb.append(getNDCG(testBPreds,testBAnswer,i))
        ndcgvalid.append(getNDCG(validPreds,validAnswer,i))
    LndcgA.append(ndcga)
    LndcgB.append(ndcgb)
    LndcgValid.append(ndcgvalid)
    y=[]
    for i in range(len(LndcgB)):
        y.append(LndcgB[i][-3])
    plotCurve(range(1,len(LndcgB)+1),y)
    
    saveFile()
print("Done!")

In [ ]:
#-----------------------------------------

epochs = 1

recallA=[]
precisionA=[]
f1A=[]
ndcgA=[]

recallB=[]
precisionB=[]
f1B=[]
ndcgB=[]

recallValid=[]
precisionValid=[]
f1Valid=[]
ndcgValid=[]

for t in range(epochs):
    train_empohs()
    global model
    testAPreds=predict(model,testADf)
    testBPreds=predict(model,testBDf)
    validPreds=predict(model,validDf)
    #recall
    rA=[]
    rB=[]
    rValid=[]
    for i in range(1,35,5):
        rA.append(getRecall(testAPreds,testAAnswer,i))
        rB.append(getRecall(testBPreds,testBAnswer,i))
        rValid.append(getRecall(validPreds,validAnswer,i))
    recallA.append(rA)
    recallB.append(rB)
    recallValid.append(rValid)

    #precision
    pA=[]
    pB=[]
    pValid=[]
    for i in range(1,35,5):
        pA.append(getPrecision(testAPreds,testAAnswer,i))
        pB.append(getPrecision(testBPreds,testBAnswer,i))
        pValid.append(getPrecision(validPreds,validAnswer,i))
    precisionA.append(pA)
    precisionB.append(pB)
    precisionValid.append(pValid)

    #f1
    f1a=[]
    f1b=[]
    f1valid=[]
    for i in range(1,35,5):
        f1a.append(getF1(testAPreds,testAAnswer,i))
        f1b.append(getF1(testBPreds,testBAnswer,i))
        f1valid.append(getF1(validPreds,validAnswer,i))
    f1A.append(f1a)
    f1B.append(f1b)
    f1Valid.append(f1valid)

    #ndcg
    ndcga=[]
    ndcgb=[]
    ndcgvalid=[]
    for i in range(1,8):
        ndcga.append(getNDCG(testAPreds,testAAnswer,i))
        ndcgb.append(getNDCG(testBPreds,testBAnswer,i))
        ndcgvalid.append(getNDCG(validPreds,validAnswer,i))
    ndcgA.append(ndcga)
    ndcgB.append(ndcgb)
    ndcgValid.append(ndcgvalid)
    y=[]
    for i in range(len(ndcgB)):
        y.append(ndcgB[i][-3])
    plotCurve(range(1,len(ndcgB)+1),y)
print("Done!")

In [ ]:
import pickle
# 存储变量的文件的名字
filename = './content/drive/MyDrive/baseline_bertBert/recallA.pkl'
# 以二进制写模式打开目标文件
f = open(filename, 'wb')
# 将变量存储到目标文件中区
pickle.dump(recallA, f)
# 关闭文件
f.close()

# 存储变量的文件的名字
filename = './content/drive/MyDrive/baseline_bertBert/precisionA.pkl'
# 以二进制写模式打开目标文件
f = open(filename, 'wb')
# 将变量存储到目标文件中区
pickle.dump(precisionA, f)
# 关闭文件
f.close()

# 存储变量的文件的名字
filename = './content/drive/MyDrive/baseline_bertBert/f1A.pkl'
# 以二进制写模式打开目标文件
f = open(filename, 'wb')
# 将变量存储到目标文件中区
pickle.dump(f1A, f)
# 关闭文件
f.close()


# 存储变量的文件的名字
filename = './content/drive/MyDrive/baseline_bertBert/ndcgA.pkl'
# 以二进制写模式打开目标文件
f = open(filename, 'wb')
# 将变量存储到目标文件中区
pickle.dump(ndcgA, f)
# 关闭文件
f.close()

# 存储变量的文件的名字
filename = './content/drive/MyDrive/baseline_bertBert/recallB.pkl'
# 以二进制写模式打开目标文件
f = open(filename, 'wb')
# 将变量存储到目标文件中区
pickle.dump(recallB, f)
# 关闭文件
f.close()

# 存储变量的文件的名字
filename = './content/drive/MyDrive/baseline_bertBert/precisionB.pkl'
# 以二进制写模式打开目标文件
f = open(filename, 'wb')
# 将变量存储到目标文件中区
pickle.dump(precisionB, f)
# 关闭文件
f.close()

# 存储变量的文件的名字
filename = './content/drive/MyDrive/baseline_bertBert/f1B.pkl'
# 以二进制写模式打开目标文件
f = open(filename, 'wb')
# 将变量存储到目标文件中区
pickle.dump(f1B, f)
# 关闭文件
f.close()

# 存储变量的文件的名字
filename = './content/drive/MyDrive/baseline_bertBert/ndcgB.pkl'
# 以二进制写模式打开目标文件
f = open(filename, 'wb')
# 将变量存储到目标文件中区
pickle.dump(ndcgB, f)
# 关闭文件
f.close()

# 存储变量的文件的名字
filename = './content/drive/MyDrive/baseline_bertBert/recallValid.pkl'
# 以二进制写模式打开目标文件
f = open(filename, 'wb')
# 将变量存储到目标文件中区
pickle.dump(recallValid, f)
# 关闭文件
f.close()

# 存储变量的文件的名字
filename = './content/drive/MyDrive/baseline_bertBert/precisionValid.pkl'
# 以二进制写模式打开目标文件
f = open(filename, 'wb')
# 将变量存储到目标文件中区
pickle.dump(precisionValid, f)
# 关闭文件
f.close()

# 存储变量的文件的名字
filename = './content/drive/MyDrive/baseline_bertBert/f1Valid.pkl'
# 以二进制写模式打开目标文件
f = open(filename, 'wb')
# 将变量存储到目标文件中区
pickle.dump(f1Valid, f)
# 关闭文件
f.close()

# 存储变量的文件的名字
filename = './content/drive/MyDrive/baseline_bertBert/ndcgValid.pkl'
# 以二进制写模式打开目标文件
f = open(filename, 'wb')
# 将变量存储到目标文件中区
pickle.dump(ndcgValid, f)
# 关闭文件
f.close()

print('initializing model...')
nDCGs=[]
best_nDCG = 0.0
model = VisualBERT(__C, pretrained_emb).to(device)
num_training_steps = np.ceil(__C.TRAIN_SIZE*k / (batch_size*grad_step)) * epochs
num_warmup_steps = int(num_training_steps*0.1)
eval_steps = num_training_steps//40*grad_step + 100
optimizer = AdamW(model.parameters(), lr=lr, correct_bias=False)
scheduler = custom_schedule(optimizer,
                            num_warmup_steps=num_warmup_steps,
                            num_training_steps=num_training_steps,
                            num_cycles=6,
                            amplitude=0.3)

epochs = 5

recallA=[]
precisionA=[]
f1A=[]
ndcgA=[]

recallB=[]
precisionB=[]
f1B=[]
ndcgB=[]

recallValid=[]
precisionValid=[]
f1Valid=[]
ndcgValid=[]

pad_len = 30
thd = 9999
n_splits = 10
for t in range(epochs):
    train_empohs()
    global model
    cls = get_cls(model, n_splits,validDf)
        # train
    testAPreds=predictCLS(model, testADf, thd, cls)
    testBPreds=predictCLS(model, testBDf, thd, cls)
    validPreds=predictCLS(model, validDf, thd, cls)
    #recall
    rA=[]
    rB=[]
    rValid=[]
    for i in range(1,35,5):
        rA.append(getRecall(testAPreds,testAAnswer,i))
        rB.append(getRecall(testBPreds,testBAnswer,i))
        rValid.append(getRecall(validPreds,validAnswer,i))
    recallA.append(rA)
    recallB.append(rB)
    recallValid.append(rValid)

    #precision
    pA=[]
    pB=[]
    pValid=[]
    for i in range(1,35,5):
        pA.append(getPrecision(testAPreds,testAAnswer,i))
        pB.append(getPrecision(testBPreds,testBAnswer,i))
        pValid.append(getPrecision(validPreds,validAnswer,i))
    precisionA.append(pA)
    precisionB.append(pB)
    precisionValid.append(pValid)

    #f1
    f1a=[]
    f1b=[]
    f1valid=[]
    for i in range(1,35,5):
        f1a.append(getF1(testAPreds,testAAnswer,i))
        f1b.append(getF1(testBPreds,testBAnswer,i))
        f1valid.append(getF1(validPreds,validAnswer,i))
    f1A.append(f1a)
    f1B.append(f1b)
    f1Valid.append(f1valid)

    #ndcg
    ndcga=[]
    ndcgb=[]
    ndcgvalid=[]
    for i in range(1,8):
        ndcga.append(getNDCG(testAPreds,testAAnswer,i))
        ndcgb.append(getNDCG(testBPreds,testBAnswer,i))
        ndcgvalid.append(getNDCG(validPreds,validAnswer,i))
    ndcgA.append(ndcga)
    ndcgB.append(ndcgb)
    ndcgValid.append(ndcgvalid)
    y=[]
    for i in range(len(ndcgB)):
        y.append(ndcgB[i][-3])
    plotCurve(range(1,len(ndcgB)+1),y)
print("Done!")

In [ ]:
import pickle
 
# 存储变量的文件的名字
filename = './content/drive/MyDrive/baseline_bertBert_lightGBM/recallA.pkl'
# 以二进制写模式打开目标文件
f = open(filename, 'wb')
# 将变量存储到目标文件中区
pickle.dump(LrecallA, f)
# 关闭文件
f.close()

# 存储变量的文件的名字
filename = './content/drive/MyDrive/baseline_bertBert_lightGBM/precisionA.pkl'
# 以二进制写模式打开目标文件
f = open(filename, 'wb')
# 将变量存储到目标文件中区
pickle.dump(LprecisionA, f)
# 关闭文件
f.close()

# 存储变量的文件的名字
filename = './content/drive/MyDrive/baseline_bertBert_lightGBM/f1A.pkl'
# 以二进制写模式打开目标文件
f = open(filename, 'wb')
# 将变量存储到目标文件中区
pickle.dump(Lf1A, f)
# 关闭文件
f.close()


# 存储变量的文件的名字
filename = './content/drive/MyDrive/baseline_bertBert_lightGBM/ndcgA.pkl'
# 以二进制写模式打开目标文件
f = open(filename, 'wb')
# 将变量存储到目标文件中区
pickle.dump(LndcgA, f)
# 关闭文件
f.close()

# 存储变量的文件的名字
filename = './content/drive/MyDrive/baseline_bertBert_lightGBM/recallB.pkl'
# 以二进制写模式打开目标文件
f = open(filename, 'wb')
# 将变量存储到目标文件中区
pickle.dump(LrecallB, f)
# 关闭文件
f.close()

# 存储变量的文件的名字
filename = './content/drive/MyDrive/baseline_bertBert_lightGBM/precisionB.pkl'
# 以二进制写模式打开目标文件
f = open(filename, 'wb')
# 将变量存储到目标文件中区
pickle.dump(LprecisionB, f)
# 关闭文件
f.close()

# 存储变量的文件的名字
filename = './content/drive/MyDrive/baseline_bertBert_lightGBM/f1B.pkl'
# 以二进制写模式打开目标文件
f = open(filename, 'wb')
# 将变量存储到目标文件中区
pickle.dump(Lf1B, f)
# 关闭文件
f.close()

# 存储变量的文件的名字
filename = './content/drive/MyDrive/baseline_bertBert_lightGBM/ndcgB.pkl'
# 以二进制写模式打开目标文件
f = open(filename, 'wb')
# 将变量存储到目标文件中区
pickle.dump(LndcgB, f)
# 关闭文件
f.close()

# 存储变量的文件的名字
filename = './content/drive/MyDrive/baseline_bertBert_lightGBM/recallValid.pkl'
# 以二进制写模式打开目标文件
f = open(filename, 'wb')
# 将变量存储到目标文件中区
pickle.dump(LrecallValid, f)
# 关闭文件
f.close()

# 存储变量的文件的名字
filename = './content/drive/MyDrive/baseline_bertBert_lightGBM/precisionValid.pkl'
# 以二进制写模式打开目标文件
f = open(filename, 'wb')
# 将变量存储到目标文件中区
pickle.dump(LprecisionValid, f)
# 关闭文件
f.close()

# 存储变量的文件的名字
filename = './content/drive/MyDrive/baseline_bertBert_lightGBM/f1Valid.pkl'
# 以二进制写模式打开目标文件
f = open(filename, 'wb')
# 将变量存储到目标文件中区
pickle.dump(Lf1Valid, f)
# 关闭文件
f.close()

# 存储变量的文件的名字
filename = './content/drive/MyDrive/baseline_bertBert_lightGBM/ndcgValid.pkl'
# 以二进制写模式打开目标文件
f = open(filename, 'wb')
# 将变量存储到目标文件中区
pickle.dump(LndcgValid, f)
# 关闭文件
f.close()

In [ ]:
# 存储变量的文件的名字
filename = './content/drive/MyDrive/baseline_bertBert_lightGBM/validPreds.pkl'
# 以二进制写模式打开目标文件
f = open(filename, 'wb')
# 将变量存储到目标文件中区
pickle.dump(validPreds, f)
# 关闭文件
f.close()

In [ ]:
!ls ./content/drive/MyDrive/baseline_bertBert/